1. Загрузите данные load_wine из sklearn.datasets. Из обучающей части исключите объекты класса 2. Обучите случайный лес, задав только гиперпараметр `n_estimators = 100`. Оцените важность признаков. Укажите название двух наиболее важных признаков.

In [5]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
wine = load_wine()

In [26]:
df = pd.DataFrame(data= np.c_[wine['data'], wine['target']],
                     columns = wine['feature_names']+['target'])
print("standart shape:", df.shape)

df = df.drop(df[df['target']==2].index)
print("dropped shape:", df.shape)

standart shape: (178, 14)
dropped shape: (130, 14)


In [27]:
from sklearn.ensemble import RandomForestRegressor

In [28]:
X = df.drop(['target'], axis=1)
y = df['target']
clas = RandomForestRegressor(n_estimators=100)
clas.fit(X, y)
#№1-2 = proline_alcohol

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [29]:
importances = clas.feature_importances_
print(df.columns.tolist()[importances.argmax()])
importances = np.delete(importances, importances.argmax())
print(df.columns.tolist()[importances.argmax()])

proline
alcohol


2. Загрузите данные load_wine из sklearn.datasets. Из обучающей части исключите объекты класса 2. Отмасштабируйте признаки, используя класс StandardScaler с гиперпараметрами по умолчанию. Обучите случайный лес, задав только гиперпараметр `n_estimators = 100`. Оцените важность признаков. Укажите название двух наиболее важных признаков.

In [30]:
from sklearn.preprocessing import StandardScaler

In [31]:
scaler = StandardScaler()
scaler.fit(df.drop(['target'], axis=1))

StandardScaler(copy = True, with_mean = True, with_std = True)

In [32]:
#масштабированеи
X = scaler.transform(df.drop(['target'], axis=1))
y = df['target']

clas_scaled = RandomForestRegressor(n_estimators = 100)
clas_scaled.fit(X, y)

importances = clas_scaled.feature_importances_
print(df.columns.tolist()[importances.argmax()])
importances = np.delete(importances, importances.argmax())
print(df.columns.tolist()[importances.argmax()])

proline
alcohol


Ниже приведена неполная реализация класса Bagging который имеет методы `fit` для обучения бэггинга над `DecisionTreeRegressor` и метод `predict` для предсказания. Допишите необходимый код, чтобы реализовать бэггинг.

используемы переменные в коде:
- `self.n_estimators`, `n_estimators` - число используемых деревьев
- `self.regressors` - список объектов класса `DecisionTreeRegressor`, к которым уже был применён метод `fit`
Данный список необъодимо заполнить в методе `fit` и использовать для предсказания в методе `predict`
- `ind`-  выбранные индексы объектов при бутстрапе

при создании объекта класса `DecisionTreeRegressor` зафиксируйте  
`random_state=0`

In [91]:
from sklearn.tree import DecisionTreeRegressor

class Bagging():   
    def __init__(self, n_estimators=10):
        self.n_estimators = n_estimators
        self.regressors = []
    
    
    def fit(self, x_train, y_train):
        for i in range(self.n_estimators):
            np.random.seed(i)
            ind = np.random.choice(np.arange(x_train.shape[0]), size = x_train.shape[0])
            self.regressors.append(DecisionTreeRegressor(random_state=0).fit(x_train.loc[ind], y_train.loc[ind]))
        
        
    def predict(self, x_test):
        res = np.zeros(x_test.shape[0])
        for clf in self.regressors:
            res += clf.predict(x_test)
        return res/self.n_estimators

Загрузите данные приложенные к заданию

In [63]:
# ==
# import pandas as pd
# from google.colab import files
# uploder = files.upload()

In [64]:
Data = pd.read_csv('data.csv', header=None)

Положим матрицу объекты-признаки в переменную `X`, а ответы в переменную `y`

In [65]:
X, y = Data.iloc[:, :100], Data.iloc[:, 100]


Положим первые 6000 объектов в обучающую часть, остальные объекты в тестовую часть

In [66]:
x_train, y_train = X[:6000], y[:6000]
x_test, y_test = X[6000:], y[6000:]

3. Обучите бэггинг на 1 дереве. Оцените качество по метрике MSE на тестовой части. Ответ разделите на 1000 и округлите до целой части по математичестким правилам округления.

In [67]:
test1 = Bagging(1)
test1.fit(x_train, y_train)

In [69]:
from sklearn.metrics import mean_squared_error
round(mean_squared_error(y_test, test1.predict(x_test))/1000)

33.0

4. Обучите бэггинг на 5 деревьях. Оцените качество по метрике MSE на тестовой части. Ответ разделите на 1000 и округлите до целой части по математичестким правилам округления.

In [92]:
test2 = Bagging(5)
test2.fit(x_train, y_train)
round(mean_squared_error(y_test, test2.predict(x_test))/1000)

15.0

5. Обучите бэггинг на 100 деревьях. Оцените качество по метрике MSE на тестовой части. В ответе укажите целую часть.

In [93]:
test3 = Bagging(100)
test3.fit(x_train, y_train)
round(mean_squared_error(y_test, test3.predict(x_test))/1000)

11.0

6. Обучите на этих же данных случайный лес, используйте гиперпараметр `n_estimators = 1`, зафиксируйте  
`random_state=0`. Оцените качество по метрике MSE на тестовой части. В ответе укажите целую часть.

In [87]:
rfregr1 = RandomForestRegressor(n_estimators=1, random_state=0)
rfregr1.fit(x_train, y_train)
round(mean_squared_error(y_test, rfregr1.predict(x_test))/1000)

35.0

8. Обучите на этих же данных случайный лес, используйте гиперпараметр `n_estimators = 5`, зафиксируйте  
`random_state=0`. Оцените качество по метрике MSE на тестовой части. В ответе укажите целую часть.

In [89]:
rfregr5 = RandomForestRegressor(n_estimators=5, random_state=0)
rfregr5.fit(x_train, y_train)
round(mean_squared_error(y_test, rfregr5.predict(x_test))/1000)

15.0

9. Обучите на этих же данных случайный лес, используйте гиперпараметр `n_estimators = 100`, зафиксируйте  
`random_state=0`. Оцените качество по метрике MSE на тестовой части. В ответе укажите целую часть.

In [90]:
rfregr100 = RandomForestRegressor(n_estimators = 100, random_state = 0)
rfregr100.fit(x_train, y_train)
round(mean_squared_error(y_test, rfregr100.predict(x_test))/1000)

11.0

10. Изучите документацию и разберитесь как посчитать Out-of-bag ошибку в RandomForestRegressor. Обучите RandomForestRegressor с гиперпараметром n_estimators=100 на обучающей части, зафиксируйте  
`random_state = 0`. Найдите Out-of-bag ошибку алгоритма. Ответ округлите до сотых.

In [84]:
rfr_out_of_bag = RandomForestRegressor(n_estimators = 100, random_state = 0, oob_score = True)
round(rfr_out_of_bag.fit(x_train, y_train).oob_score_, 2)

0.75